Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## This cell configures the spark session - Do not change

In [27]:
%%configure -f
{
"conf": {
     "spark.rpc.message.maxSize": 1024,
     "spark.kryoserializer.buffer.max": "256m"
   }
}

StatementMeta(, 45, -1, Finished, Available)

## These are the parameters that need to be changed to your values

In [28]:
# The input file directory
input_directory = "abfss://share@datadiscoverypipeline.dfs.core.windows.net/bbcsports"
# The output directory where the output file will be written to
output_directory = 'abfss://share@datadiscoverypipeline.dfs.core.windows.net/videos_outputs/'
# The name of the output file
output_filename = 'bbc_text_summarisation.csv'
# If this is set to True then the Coalesce notebook will need to be run to merge the partition files into a single file
LOW_MEMORY_MODE = True

# AML Experiment tracking

# Azure SubscriptionId
subscription_id=""
# AzureML Workspace Resource Group
resource_group=""
# AzureML Workspace Name
workspace_name=""

StatementMeta(DataDiscovery, 45, 1, Finished, Available)

## Track the Experiment in Azure ML

In [ ]:
from azureml.core import Workspace, Experiment, Run
import mlflow

ws = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name)    
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
experiment_name = f"({mssparkutils.runtime.context['notebookname']}_{str(mssparkutils.env.getJobId())})"
mlflow.set_experiment(experiment_name)
mlflow.log_param("input_filename", input_filename)
mlflow.log_param("output_directory", output_directory)
mlflow.log_param("output_filename", output_filename)
mlflow.log_param("LOW_MEMORY_MODE", LOW_MEMORY_MODE)
params = {
    "sparkpool": mssparkutils.runtime.context['sparkpool'],
    "workspace": mssparkutils.runtime.context['workspace'],
    "notebookname": mssparkutils.runtime.context['notebookname'],
    "isForPipeline": mssparkutils.runtime.context['isForPipeline'],
    "pipelinejobid": mssparkutils.runtime.context['pipelinejobid']
}

mlflow.log_params(params)
mlflow.pyspark.ml.autolog()

In [34]:
from  pyspark.sql.functions import input_file_name
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, IntegerType
from pyspark.sql.functions import spark_partition_id
from pyspark.ml.linalg import Vectors, VectorUDT
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import os

df = spark.read.text(input_directory, wholetext=True)   
df = df.withColumn("filename", input_file_name())

StatementMeta(DataDiscovery, 45, 7, Finished, Available)

In [35]:
import numpy as np
import torch


def pegasus_summarise_text(df):

    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

    text_abs = []
    text_abs = df.value.values.tolist()

    summaries = []
    for c in text_abs:
        inputs = tokenizer(c, max_length=512, return_tensors="pt", truncation=True)
        summary_ids = model.generate(inputs["input_ids"])
        summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        summaries.append(summary)

    return_df = (
        df[["filename"]]
        .assign(value=summaries)
    )
    return return_df

StatementMeta(DataDiscovery, 45, 8, Finished, Available)

In [36]:
summary_schema = StructType(
    [
        StructField("filename", StringType(), True),
        StructField("value", StringType(), True)
    ]
)

summary_df = (
    df
    .groupBy("filename")
    .applyInPandas(pegasus_summarise_text, summary_schema)
)


StatementMeta(DataDiscovery, 45, 9, Finished, Available)

In [37]:
if LOW_MEMORY_MODE:
    summary_df.write.mode('overwrite').options(header='true').csv(os.path.join(output_directory, output_filename))
else:
    summary_df.coalesce(1).write.mode('overwrite').options(header='true').csv(os.path.join(output_directory, output_filename))

mlflow.pyspark.ml.mlflow.end_run()

StatementMeta(DataDiscovery, 45, 10, Finished, Available)

# Add Azure Cognitive Search

## Add Search Parameters

In [ ]:
# Azure Search Admin Key
search_admin_key = ""
# The name of the search service
search_service_name = ""
# The Azure Search Query Key
search_query_key = ""

In [ ]:
from synapse.ml.cognitive import *
from pyspark.sql.functions import monotonically_increasing_id, lit

df = df.drop("_c0")

(
    df.withColumn("key", monotonically_increasing_id().cast("string"))
    .withColumn("SearchAction", lit("upload"))
    .writeToAzureSearch(
        subscriptionKey=search_admin_key,
        actionCol="SearchAction",
        serviceName=search_service_name,
        indexName=experiment_name,  # Defaults to the notebook name
        keyCol="key",
    )
)

## Search the generated Azure Search Index

In [ ]:
import requests
from IPython.display import display, Markdown

term_to_search_for = "covid"

url = "https://{}.search.windows.net/indexes/{}/docs/search?api-version=2019-05-06".format(
    search_service_name, experiment_name

)
jdata = requests.post(url, json={"search": term_to_search_for}, headers={"api-key": search_query_key}).json()

for doc in jdata['value']:
    display(Markdown(f'**Search Score {doc["@search.score"]}** Document {doc["filename"]}'))
    display(Markdown(f'{doc["text"]}'))


# Implement Semantic Search

1) [Enable Semantic Search](https://docs.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search) on your search instance

2) [Configure Semantic Search](https://docs.microsoft.com/en-us/azure/search/semantic-how-to-query-request?tabs=semanticConfiguration%2Cportal#create-a-semantic-configuration)

In [ ]:
term_to_search_for = "Whose thumb was fractured?"

url = "https://{}.search.windows.net/indexes/{}/docs/search?api-version=2021-04-30-Preview".format(
    search_service_name, experiment_name
)
jdata = requests.post(url, json={"search": term_to_search_for, "queryType": "semantic", "semanticConfiguration": "config", "queryLanguage": "en-us", "answers": "extractive|count-3",
"captions": "extractive|highlight-true",  "highlightPreTag": "<mark>","highlightPostTag": "</mark>"}, headers={"api-key": search_query_key}).json()

for doc in jdata['value']:
    display(Markdown(f'**Search Score {doc["@search.score"]}** **Search rerankerScore Score {doc["@search.rerankerScore"]}** Document {doc["filename"]}'))
    display(Markdown(f'@search.captions {doc["@search.captions"]}'))
